In [1]:
from __future__ import division
import pdb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  #
sns.set_style("white")
sns.set_style("ticks")

#import meanderpyPM_MTD_Jan2021Amazon as mp 
#import meanderpyPM_MTD_Jan2021 as mp
import meanderpyPM_MTD_RandomStacking4_Model as mp
import pickle as pickle
np.set_printoptions(threshold=100000)
import os as os
from imp import reload
%matplotlib qt

%load_ext autoreload


### LOAD PREVIOUS MODEL

In [2]:
##LOAD IN CENTERLINE FOR TRANSLATION
chb=pickle.load(open("Alpak_ChannelBelt_May2022", "rb")) ##LOAD IN CENTERLINE FOR AMAZON

no_agg=0 
low_agg=0
med_agg=31
high_agg=31

#chb = __init__(self, channels, cutoffs, cl_times, cutoff_times, MTDs, MTD_times):

In [ ]:
##Look at channel times so you can choose one to be an MTD
chb.cl_times

In [ ]:
## ADD MTD INTO MODEL

chb.MTD_times.append(5168)

### INPUT PARAMETERS

In [3]:
#MCHARGUE ORGANIZED BEHAVIOUR HARD CODED INTO .PY FILE

W = 600 #600.0                    # channel width (m)
D = 60#60.0                     # channel depth (m)
pad = 5                   # padding (number of nodepoints along centerline)
#deltas = W/2                # sampling distance along centerline
deltas=W/2
saved_ts = 10            # which time steps will be saved         
Cf = 0.048                # dimensionless Chezy friction factor
#crdist = 2.5*W               # threshold distance at which cutoffs occur
crdist = 1*W 
kl = 8/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-11               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*365*24*60*60.0 # 0.2 years
dens = 1000                  # density of water (kg/m3)
n_bends = 25                 # approximate number of bends you want to model
Sl = 0.000 # initial slope (matters more for submarine channels than rivers)
#Sl=0
bthc = 0.25*D                 # Max thickness of deposited channel sand
#t1 = 500                    # time step when incision starts
#t2 = 700                    # time step when lateral migration starts
#t3 = 1400                    # time step when aggradation starts
#aggr_factor = 4e-9           # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3)

print(f"Alpha ={1*2*Cf/D}")

print(f"Model on {np.arctan(Sl)*180/np.pi} degree incline")

Alpha =0.0016
Model on 0.0 degree incline


### INPUT TRAJECTORY

In [4]:
#dzinput=(0,0,-0.4,-0.4,-0.4,-0.4,0,0,0.3,0.3,0.35,0.35)  
   
#timeinput=(0,2200,2201,2540,2541,2880,2881,3700,3701,3780,3781,4480)

dzinput=(0.0,0.0) 
   
timeinput=(0,99999)

#######AMAZON########
#dzinput=(0.3,0.3) 
   
#timeinput=(0,560)   
#######################

######BENINTEST######
#dzinput=(0,0,-0.4,-0.4,0,0,0.3,0.3,0.35,0.35)  
   
#timeinput=(0,2200,2201,2880,2881,3700,3701,3780,3781,4200)

#######BENINLIKE - PAPER ONE#######################################
#dzinput=(0,0,-0.4,-0.4,0,0,0.3,0.3,0.3,0.3)  
   
#timeinput=(0,2200,2201,2880,2881,3700,3701,3780,3781,4480)
###############################################################


#######ZAIRELIKE - PAPER ONE#######################################
#dzinput=(0,0,-0.4,-0.4,0,0)  
   
#timeinput=(0,2200,2201,2880,2881,3700)
###############################################################

In [5]:
reload(mp)
nit = np.max(timeinput)  # number of iterations

dz_trajectory, Z_trajectory, CLZjump,CLZjumpbthc = mp.Compute_Trajectory(nit,saved_ts,timeinput,dzinput,bthc)


In [ ]:
print(dz_trajectory[1000])

### INITIAL CHANNEL BELT AND HK

In [ ]:
reload(mp)
ch,sinuosity = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
chb = mp.ChannelBelt(channels=[ch],cutoffs=[],cl_times=[0.0],cutoff_times=[],MTDs=[],MTD_times=[]) # create channel belt object

In [ ]:
channel=chb.channels[-1]

In [ ]:
print(sinuosity)

In [ ]:
#chb.migrate_PM_RANDOM(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,dz_trajectory,D) # channel migration

no_agg=0 #biut fo erosion (no real vertical movement) MCHargue Stage 1
low_agg=1
med_agg=30
high_agg=30 #last few channels are highly aggradaitonal and organized stacking but then abandoned

#no_agg=10 
#low_agg=20
#med_agg=30
#high_agg=30

chb.migrate_PM_RANDOM(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,dz_trajectory,no_agg,low_agg,med_agg,high_agg,D)

fig = chb.plot('strat',1000000,1000000,end_time=99999999,n_channels=(len(chb.cutoffs) + len(chb.channels)))# plotting #green numbers age at which stuff gets covered by vegetation

In [ ]:
fig = chb.plot('strat',1000000,1000000,end_time=99999999,n_channels=(len(chb.cutoffs) + len(chb.channels)))# plotting #green numbers age at which stuff gets covered by vegetation

In [ ]:
#LOAD IN FinalChanel
file=open('U_kl8_FinalChannel', 'rb') #LOAD IN ZAIRE-LIKE CHANNEL BELT MODEL
#chb=pickle.load(file)
chb.channels[-1]


In [ ]:
#SAVE CHANNEL BELT INSTANCE
ChannelBelt=chb
pickle.dump(ChannelBelt,open("Alpak_ChannelBelt_May2022", "wb"),pickle.HIGHEST_PROTOCOL)

In [6]:
print(f"The number of cut-offs = {len(chb.cutoffs)}")
print(f"The number of centerlines = {len(chb.channels)}")
#print(f"The number of MTD's = {len(chb.MTDs)}")
print(f"Bed Thickness = is {bthc} meters")
print(chb.cl_times)
print(len(chb.cl_times))
print(f"MTD times = {chb.MTD_times}")

The number of cut-offs = 0
The number of centerlines = 30
Bed Thickness = is 15.0 meters
[0.0, 1196.0, 2266.0, 3380.0, 4574.0, 5634.0, 6648.0, 7804.0, 8930.0, 10138.0, 11240.0, 12328.0, 13422.0, 14450.0, 15554.0, 16698.0, 17756.0, 18878.0, 19920.0, 21010.0, 22222.0, 23246.0, 24400.0, 25506.0, 26736.0, 27792.0, 28824.0, 29988.0, 31010.0, 32066.0]
30
MTD times = []


In [ ]:
plt.figure()
for i in range(len(chb.channels)):
    x,y,z,dx,dy,dz,ds,s = mp.resample_centerline(chb.channels[i].x, chb.channels[i].y, chb.channels[i].z, deltas)
    z = np.minimum.accumulate(-z[::-1])
    z = -z[::-1]
    plt.plot(s, z, 'k')

In [ ]:
for i in range(len(chb.channels)):
    chb.channels[i].z = np.minimum.accumulate(-chb.channels[i].z[::-1])
    chb.channels[i].z = -chb.channels[i].z[::-1]

In [ ]:
len(xmin)

### BUILD 3D MODEL

In [7]:
#%matplotlib qt
starttime=int(chb.cl_times[1])
endtime=chb.cl_times[-1]
dx=30
levee_width=12000
h = 1.0*D # channel depth
w = 1.0*W # channel width
dcr=bthc
#bth = bthc*np.ones(((len(chb.cl_times[0:]),)))
bth=bthc
delta_s=deltas
TurbidityCurrentHeight = 150
CLZjump=CLZjump
#h_mud = 3.0*np.ones((len(chb.cl_times[0:]),))
h_mud = 3.0#THIS IS A DUMMY VARIABLE

In [8]:
reload(mp)

#Amazon

#xmin=15000
#xmax=21000ccc


#ymin=-2000
#ymax=2000

###Benin and Zaire###
#xmin=35000
#xmax=45000
#ymin=-5000
#ymax=5000

xmin=41000
xmax=51000
ymin=-5000
ymax=5000

#chb_3d, xmin, xmax, ymin, ymax,indices,dists,zmaps = chb.build_3d_model_PM_MTD_RANDOM('submarine',h_mud=h_mud,levee_width=levee_width,h=h,w=w,bth=bth,
#                            dcr=dcr,dx=dx,delta_s=delta_s,starttime=chb.cl_times[0],endtime=chb.cl_times[-1],xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,CLZjump=CLZjump,TurbidityCurrentHeight=TurbidityCurrentHeight)

chb_3d, xmin, xmax, ymin, ymax,indices,dists,zmaps = chb.build_3d_model_PM_MTD_RANDOM2('submarine',h_mud=h_mud,levee_width=levee_width,h=h,w=w,bth=bth,
                            dcr=dcr,dx=dx,delta_s=delta_s,starttime=chb.cl_times[0],endtime=chb.cl_times[-1],xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,CLZjump=CLZjump,TurbidityCurrentHeight=TurbidityCurrentHeight,no_agg=no_agg,low_agg=low_agg,med_agg=med_agg,high_agg=high_agg)




Percent: [###################-] 96.55172413793103%  

In [ ]:
xsec_start=int(0)
xsec_end=int((xmax-xmin)/dx)
Ztrajectory=[1,1]
Msb_MAXEntireBelt, Msb_MINEntireBelt, Msb_MEANEntireBelt, CrossSecBeltAveragedStratMobility_OVERALL ,Normalised_Belt_Heights ,Normalised_Belt_Widths, Belt_Aspect_Ratio_OVERALL=chb_3d.Msb_EntireChannelBelt(xsec_start,xsec_end,w,h,Ztrajectory)


In [ ]:
chb_3d.Plot_Msb_ChannelBelt(Msb_MAXEntireBelt, Msb_MINEntireBelt, Msb_MEANEntireBelt, CrossSecBeltAveragedStratMobility_OVERALL,Normalised_Belt_Heights ,Normalised_Belt_Widths,Belt_Aspect_Ratio_OVERALL)

In [ ]:
#CHECK topographic evolution

for i in range(0,10):
    plt.figure()
    plt.plot(chb_3d.topo[:,300,i])

In [ ]:
#Check Channel Heights

In [ ]:
McHargueModel_chb_3d=chb_3d

#os.chdir('/Users/paulmorris/Documents/JoshuaChannel/Post Paradigm/Python 3/August 2020 Centerlines/')
#pickle.dump(JoshuaModel_chb_3d, open("JoshuaModel_chb_3d", "wb"),pickle.HIGHEST_PROTOCOL)

os.chdir('/Users/paulmorris/Documents/Flow Sim Paper/Fall 2021 New Stuff/Python Models/')

with open("McHargueModel_chb_3d_Fall2021", "wb") as file_:
    pickle.dump(McHargueModel_chb_3d, file_, -1)

In [ ]:
ci=20
chb_3d.plot_top_and_bottom_surfaces(ci)


In [9]:
fig1,fig2,fig3 = chb_3d.plot_xsection_PM_MTD(200, 5,indices)

##### np.shape(chb_3d.strat)

In [ ]:
len(chb.channels)

### PROPERTY MODELING AND EXPORT 

In [ ]:
ny, nx, nz = np.shape(chb_3d.strat)
h = D # channel depth
porosity = np.zeros((ny-1,nx-1,nz-1)) # property grids have to be one element smaller than the actual grid

poro_max=0.35
trend=8

indices=np.where(np.in1d(chb.cl_times, chb.MTD_times))[0] #finds MTD times

for i in range(zmaps.shape[2]): # only working with channel sands
    
    if i in indices: #This sets MTD fills to be a certain poro perm value
        porosity[:,:,3*i+2]=0.01
        
    else:
        hat = np.abs(chb_3d.topo[:,:,4*i+1] - zmaps[:,:,i] + h) # height above thalweg
        th = chb_3d.topo[:,:,4*i+4] - chb_3d.topo[:,:,4*i+3] # thickness of channel deposit
        hat[th==0] = 1000 # set a large HAT value where thickness is zero (we want zero porosity here)
        #t = 0.25*(hat[1:,1:]+hat[1:,:-1]+hat[:-1,1:]+hat[:-1,:-1]) # average HAT for porosity grid
        t = 0.25*(hat[1:,1:]+hat[1:,:-1]+hat[:-1,1:]+hat[:-1,:-1]) # average HAT for porosity grid
        t = t-np.min(t)

        #t[t>30] = 30 
        t[t>trend] = trend #lower number means porosity reduces more rapidly into margin

        # porosity is a function of elevation (the higher the elevation, the lower the porosity):
        #t = 0.35 - 0.35*(t/30.0)
        t = poro_max - poro_max*(t/trend) #Max porosity is 0.25

        porosity[:,:,3*i+2] = t # assign porosity
    
#PAUL CODE - Turn cells either side in model ON or OFF

#porosity[:,:,:]=0

#porosity[:,:,:]=0


porosity[porosity>poro_max] = poro_max
permeability = 10**(17*porosity-3)
permeability[porosity==0] = 0

In [ ]:
# QC porosity volume

plt.figure()
plt.imshow(porosity[:,:,2])
plt.colorbar()

In [ ]:
# QC porosity volume

plt.figure()
plt.imshow(porosity[:,:,305])
plt.colorbar()

In [ ]:
# QC porosity volume

plt.figure()
plt.imshow(porosity[:,:,272])
plt.colorbar()

In [ ]:
np.shape(porosity)

In [ ]:
os.chdir('/Users/paulmorris/Documents/Flow Sim Paper/Fall 2021 New Stuff/Eclipse Models')

In [ ]:
def write_eclipse_grid(strat,porosity,permeability,dx,fname):
    '''function for exporting an Eclipse file ('.grdecl' format)
    from an array of stratigraphic surfaces ('strat') and an array of 
    porosity ('porosity'); (additional 'keywords' like ACTNUM and SATNUM 
    can be added the same way as porosity)

    ordering of cornerpoints for first (uppermost) surface in Eclipse:
    ----------------------------------------
    | 1        2 | 3        4 | 5        6 | 
    |            |            |            |
    | 7        8 | 9       10 | 11      12 |
    ----------------------------------------
    | 13      14 | 15      16 | 17      18 |
    |            |            |            |
    | 19      20 | 21      22 | 23      24 |
    ----------------------------------------

    Inputs:
    strat - stratigraphic surfaces (outputs from channel model)
    porosity - porosity grid
    dx - gridcell size (in meters)
    fname - filename of the Eclipse file to be written
    '''

    # these swaps have to be done because the logic below was written for this ordering of the axes
    surfs = np.swapaxes(strat, 0, 2) 
    surfs = np.swapaxes(surfs, 1, 2)
    porosity = np.swapaxes(porosity, 0, 2)
    porosity = np.swapaxes(porosity, 1, 2)
    permeability = np.swapaxes(permeability, 0, 2)
    permeability = np.swapaxes(permeability, 1, 2)
    
    nz,ny,nx = np.shape(surfs);
    nz=nz-1 # number of cells in z direction
    ny=ny-1 # number of cells in y direction
    nx=nx-1 # number of cells in x direction

    dy=dx  # size of cells in x and y directions

    print('creating cornerpoint array(ZCORN)')
    zcorn = np.zeros((8*nx*ny*nz,))
    for k in range(nz):
        # write cornerpoints for the top of layer 'k':
        surf = np.squeeze(surfs[k,:,:])
        zc = np.zeros((2*ny,2*nx))
        zc[::2,::2] = surf[:-1,:-1]
        zc[1::2,1::2] = surf[1:,1:]
        zc[1::2,::2] = surf[1:,:-1]
        zc[::2,1::2] = surf[:-1,1:]
        zc = np.reshape(zc,(1,4*nx*ny))
        zcorn[(2*k)*4*nx*ny : (2*k+1)*4*nx*ny] = zc;
        
        # write cornerpoints for the bottom of layer 'k':
        surf = np.squeeze(surfs[k+1,:,:])
        zc = np.zeros((2*ny,2*nx));
        zc[::2,::2] = surf[:-1,:-1]
        zc[1::2,1::2] = surf[1:,1:]
        zc[1::2,::2] = surf[1:,:-1]
        zc[::2,1::2] = surf[:-1,1:]
        zc = np.reshape(zc,(1,4*nx*ny))
        zcorn[(2*k+1)*4*nx*ny : (2*k+2)*4*nx*ny] = zc
    zcorn = np.reshape(zcorn, (int(len(zcorn)/8), 8))
    zcorn = 100 * zcorn # convert meters to centimeters

    print('creating pillar matrix (COORD)')
    coord = np.zeros(((nx+1)*(ny+1),6));
    for j in range(ny+1):
        for i in range(nx+1):
            coord[j*(nx+1)+i,0] = i*dx
            coord[j*(nx+1)+i,1] = j*dy
            coord[j*(nx+1)+i,2] = surfs[0,j,i] 
            coord[j*(nx+1)+i,3] = i*dx
            coord[j*(nx+1)+i,4] = j*dy
            coord[j*(nx+1)+i,5] = surfs[-1,j,i]
    coord = 100 * coord # convert meters to centimeters

    print('creating porosity array (PORO)')
    poro = np.zeros((nx*ny*nz,))
    [i,j,k] = np.meshgrid(np.arange(nx),np.arange(ny),np.arange(nz))
    ind1 = k*nx*ny + j*nx+i 
    ind2 = np.ravel_multi_index((k,j,i),(nz,ny,nx))
    poro[ind1] = porosity.flatten()[ind2]
    if np.mod(len(poro),8)==0: # if length of porosity array is a multiple of 8
        poro_1 = np.reshape(poro, (int(len(poro)/8), 8))
    else:
        poro_1 = np.reshape(poro[:-np.mod(len(poro),8)], (int(len(poro)/8), 8))

    print('creating permeability array (PERM)')
    perm = np.zeros((nx*ny*nz,))
    perm[ind1] = permeability.flatten()[ind2]
    if np.mod(len(perm),8)==0: # if length of porosity array is a multiple of 8
        perm_1 = np.reshape(perm, (int(len(perm)/8), 8))
    else:
        perm_1 = np.reshape(perm[:-np.mod(len(perm),8)], (int(len(perm)/8), 8))
    
    # write file:
    fid = open(fname, 'a')
    fid.write('SPECGRID\n')
    fid.write('%d %d %d' %(nx, ny, nz) + ' 1 F /\n')
    fid.write('COORD\n')
    print('writing pillars...')
    for i in range(coord.shape[0]):
        fid.write('%6.3f %6.3f %6.3f %6.3f %6.3f %6.3f\n' %tuple(coord[i,:]))
    fid.write('/\n')
    fid.write(' ')
    fid.write('\n')
    fid.write('ZCORN\n')
    print('writing zcorns...')
    for i in range(zcorn.shape[0]):
        fid.write('%6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f %6.3f\n' %tuple(zcorn[i,:]))
    fid.write('/\n')
    fid.write(' ')
    fid.write('\n')
    fid.write('PORO\n')
    print('writing porosity...')
    for i in range(poro_1.shape[0]):
        fid.write('%6.4f %6.4f %6.4f %6.4f %6.4f %6.4f %6.4f %6.4f\n' %tuple(poro_1[i,:]))
    if np.mod(len(poro),8)!=0: # if length of porosity array is not a multiple of 8
        for i in range(np.mod(len(poro),8)):
            fid.write('%6.4f ' %poro[-np.mod(len(poro),8):][i])
        fid.write('\n')
    fid.write('/\n')
    fid.write(' ')
    fid.write('\n')
    fid.write('PERMX\n')
    print('writing permeability...')
    for i in range(perm_1.shape[0]):
        fid.write('%6.4f %6.4f %6.4f %6.4f %6.4f %6.4f %6.4f %6.4f\n' %tuple(perm_1[i,:]))
    if np.mod(len(perm),8)!=0: # if length of porosity array is not a multiple of 8
        for i in range(np.mod(len(perm),8)):
            fid.write('%6.4f ' %perm[-np.mod(len(perm),8):][i])
        fid.write('\n')
    fid.write('/\n')
    fid.close()

In [ ]:
#dx = 30.0
write_eclipse_grid(-1*(chb_3d.strat[:,:,::-1]-500), porosity[:,:,::-1], permeability[:,:,::-1], dx, 'Alpak_w600_Feb22.grdecl')

### SHOW 3D MODEL MAYAVI

In [ ]:
import blockdiagramPaul as bd
from mayavi import mlab 
reload(bd)
%matplotlib qt

mlab.figure(bgcolor=(1,1,1)) 
ve = 5.0 # vertical exaggeration
scale = 0.1 # scaling of diagram (important for 3D printing)
strat_switch = 1 # equals 1 if you want stratigraphy displayed on the sides
layers_switch = 0 # equals 1 if you want stratigraphic boundaries displayed on the sides
contour_switch = 1 # equals 1 if you want contours displayed on the top surface
dx = 30.0 # cell size for display
bottom = np.min(chb_3d.strat) - 1.5 # elevation of bottom side of diagram
color_mode = 'facies' # determines how the stratigraphy will be colored; can be 'property', 'facies', or 'time'
colors = [[0.5,0.5,0.5],[0.9,0.9,0],[0.5,0.5,0.5]] # colors for 'facies' display
line_thickness = 1.0 # thickness of lines if 'layers_switch' is 1
gap = 120 # distance between exploded blocks (if any; in number of gridcells)
h = h # channel depth (m)
nx = 1 # number of blocks in x direction
ny = 1
export = 0
ci=20
bd.create_exploded_view(chb_3d.strat,chb_3d.facies,chb_3d.topo,h,nx,ny,gap,dx,ve,scale,strat_switch,
                        layers_switch,contour_switch,color_mode,colors,line_thickness,bottom,export,ci)

In [ ]:
gap=120
bd.create_exploded_view(chb_3d.strat,chb_3d.facies,chb_3d.topo,h,2,2,gap,dx,ve,scale,strat_switch,
                        layers_switch,contour_switch,color_mode,colors,line_thickness,bottom,export)

#### Create fence diagram

In [ ]:
mlab.figure(bgcolor=(1,1,1))
bd.create_fence_diagram(chb_3d.strat,chb_3d.facies,chb_3d.topo,h,4,2,gap,dx,ve,scale,layers_switch,color_mode,colors,line_thickness,bottom,export)

# #Animation

In [ ]:
import blockdiagramPaul as bd
from mayavi import mlab 

mlab.figure(bgcolor=(1,1,1),size=(2500,1500))
##ISOMETRIC###
#mlab.view(azimuth=45,elevation=54.7356, distance=2755.8, focalpoint=[  499.        ,  499.      ,   28.07007721]) 

#PLANVIEW
#mlab.view(azimuth=270,elevation=0, distance=27365.6, focalpoint=[  4990.        ,  4990.      ,   281.07007721]) 

#XSECTIONVIEW
mlab.view(azimuth=-90,elevation=90, distance=1000, focalpoint=[  4990.        ,  4990.      ,   281.07007721]) 


ve = 5.0
offset = 0
scale = 1.0
strat_switch = 1
layers_switch = 1
contour_switch = 1
dx = 20.0

color_mode = 'facies' # determines how the stratigraphy will be colored; can be 'property', 'facies', or 'time'
#colors = [[0.5,0.5,0.5],[0.9,0.9,0],[0.5,0.5,0.5]] # colors for 'facies' display
colors = [[1.0,1.0,0],[0.5,0.5,0.5],[0.5,0.5,0.5]]
line_thickness = 1.0 # thickness of lines if 'layers_switch' is 1
gap = 120 # distance between exploded blocks (if any; in number of gridcells)
h = h # channel depth (m)
nx = 1 # number of blocks in x direction
ny = 1
export=0

x1 =13500/dx
x2 =17500/dx

i=25 #i=starttime
for j in range(26,111): #j= endtime
    
    mlab.clf()

    chb_3d, xmin, xmax, ymin, ymax,indices,dists,zmaps = chb.build_3d_model_PM_MTD('submarine',h_mud=h_mud,levee_width=levee_width,h=h,w=w,bth=bth,
                            dcr=dcr,dx=dx,delta_s=delta_s,starttime=chb.cl_times[i],endtime=chb.cl_times[j],xmin=xmin,xmax=xmax,ymin=ymin,ymax=ymax,CLZjump=CLZjump,TurbidityCurrentHeight=TurbidityCurrentHeight)
    
    bottom = np.min(chb_3d.strat) - 1.5
    
    #topo, strat, xmin, xmax, ymin, ymax, X_pix, Y_pix, S_pix, Z_pix = hk.ch_punct_migr(h_mud,levee_width,X,Y,Z,dcr,bth,h,w,dx,Sl,cutoff_times,saved_cl_times,XC,YC,ZC,deltas,saved_ts,starttime,endtime,'hello',CLZjump)
    bd.create_exploded_view(chb_3d.strat,chb_3d.facies,chb_3d.topo,h,nx,ny,gap,dx,ve,scale,strat_switch,
                        layers_switch,contour_switch,color_mode,colors,line_thickness,bottom,export,20)
    
    
    
    #x1 =13500/dx
    #x2 =23500/dx
    #y1=5000/dx
    #y2=-5000/dx
    #offset = 0

    #bd.create_block_diagram(strat[y1:y2,x1:x2,:],dx,ve,offset,scale,10.0,strat_switch,contour_switch,bottom)

    #bd.add_stratigraphy_to_block_diagram(strat[y1:y2,x1:x2,:],dx,ve,offset,scale,layers_switch)
    
    #UPDATE CAMERA EACH TIME IN CASE MOVED
    #mlab.view(azimuth=45,elevation=54.7356, distance=2755.8, focalpoint=[  499.        ,  499.      ,   28.07007721]) 

    
    #MAKE SURE THIS IS CORRECT!!!!!!
    #fname = '/Users/paulmorris/Documents/Paper 1/PostJake Python Models/Python3_MayaviVideos/From Flow Sim (Paper 3 Stuff)/BENIN/Benin_Isometric/Benin_IsometricView{}.bmp'.format(j)
    fname = '/Users/paulmorris/Documents/Flow Sim Paper/Fall 2021 New Stuff/McHargue Animation/McHargue_XsecView{}.bmp'.format(j)

    mlab.savefig(fname)
    
    ##ISOMETRIC###
    #mlab.view(azimuth=45,elevation=54.7356, distance=2755.8, focalpoint=[  499.        ,  499.      ,   28.07007721]) 

    #PLANVIEW
    #mlab.view(azimuth=270,elevation=0, distance=27365.6, focalpoint=[  4990.        ,  4990.      ,   281.07007721]) 

    #XSECTIONVIEW
    mlab.view(azimuth=-90,elevation=90, distance=1000, focalpoint=[  4990.        ,  4990.      ,   281.07007721]) 

    
    #mlab.show()